<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [73]:
import pandas as pd
import dill
import joblib
import sys
from sklearn.linear_model import LogisticRegression

from sklearn.feature_selection import SelectFromModel
from P7_functions import CustTransformer
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline

In [10]:
# Load Data train et test sets
with open('data\\dict_cleaned_samp.pkl', 'rb') as file:
    dict_cleaned = dill.load(file)

# Best model and best threshold
with open('model\\bestmodel_thresh.pkl', 'rb') as file:
    best_model, thresh = dill.load(file)

In [44]:
# Nouveaux pickles avec joblib

import joblib
with open('data\\bestmodel_joblib.pkl', 'wb') as file:
    joblib.dump(best_model, file)

In [ ]:
# # unpickle with pickle

# with open('data\\bestmodel_thresh_pickle.pkl', 'rb') as file:
#     best_model, thresh = pickle.load(file)
# with open('data\\dict_cleaned_samp_pickle.pkl', 'rb') as file:
#     dict_cleaned = pickle.load(file)

In [48]:
#  nouveau modèle tout simple
lg = LogisticRegression()
lg.fit(X_tr_featsel, y_train)

LogisticRegression()

In [53]:
best_model

Pipeline(steps=[('preproc',
                 CustTransformer(strat_high_card='loo', thresh_card=8)),
                ('featsel',
                 SelectFromModel(estimator=LGBMClassifier(), max_features=64)),
                ('clf',
                 LGBMClassifier(extra_trees=True, is_unbalance=True,
                                learning_rate=0.034600991954157015, max_depth=3,
                                n_estimators=1071, num_leaves=85,
                                random_state=14, reg_alpha=0.001815601938420407,
                                reg_lambda=0.1168368098621301))])

In [75]:
from class_transformer import CustTransformer

In [76]:
# recreate the steps of the pipeline from zero
preproc_step = CustTransformer(strat_high_card='loo', thresh_card=8)
featsel_step = SelectFromModel(estimator=LGBMClassifier(), max_features=64)
clf_step = LGBMClassifier(extra_trees=True, is_unbalance=True,
                                learning_rate=0.034600991954157015, max_depth=3,
                                n_estimators=1071, num_leaves=85,
                                random_state=14, reg_alpha=0.001815601938420407,
                                reg_lambda=0.1168368098621301)

In [54]:
# preproc_step = best_model.named_steps['preproc']
# featsel_step = best_model.named_steps['featsel']
# clf_step = best_model.named_steps['clf']

In [57]:
# dump separately the three steps of the pipeline
with open('data\\preproc_step.pkl', 'wb') as file:
    joblib.dump(preproc_step, file)
with open('data\\featsel_step.pkl', 'wb') as file:
    joblib.dump(featsel_step, file)
with open('data\\clf_step.pkl', 'wb') as file:
    joblib.dump(clf_step, file)

In [29]:
# unpickle with joblib

with open('data\\bestmodel_thresh_joblib.pkl', 'rb') as file:
    best_model, thresh = joblib.load(file)
with open('data\\dict_cleaned_samp_joblib.pkl', 'rb') as file:
    dict_cleaned = joblib.load(file)

In [78]:
path = os.path.join('data', 'X_train.csv')
X_train = pd.read_csv(path, index_col='SK_ID_CURR')
path = os.path.join('data', 'y_train.csv')
y_train = pd.read_csv(path, index_col='SK_ID_CURR')
path = os.path.join('data', 'X_test.csv')
X_test = pd.read_csv(path, index_col='SK_ID_CURR')

In [79]:
X_train.shape, y_train.shape, X_test.shape

((20000, 70), (20000, 1), (5000, 70))

In [80]:
best_model = Pipeline([('preproc', preproc_step),
	                   ('featsel', featsel_step),
                       ('clf', clf_step)])

In [81]:
best_model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\ProgramData\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Pipeline(steps=[('preproc',
                 CustTransformer(strat_high_card='loo', thresh_card=8)),
                ('featsel',
                 SelectFromModel(estimator=LGBMClassifier(), max_features=64)),
                ('clf',
                 LGBMClassifier(extra_trees=True, is_unbalance=True,
                                learning_rate=0.034600991954157015, max_depth=3,
                                n_estimators=1071, num_leaves=85,
                                random_state=14, reg_alpha=0.001815601938420407,
                                reg_lambda=0.1168368098621301))])

In [82]:
# # compute processed data (first steps of the best_model pipeline)
# X_train = dict_cleaned['X_train']
# y_train = dict_cleaned['y_train']
# X_test = dict_cleaned['X_test']

# # split the steps of the best pipeline
# preproc_step = best_model.named_steps['preproc']
# featsel_step = best_model.named_steps['featsel']
# clf_step = best_model.named_steps['clf']

# compute the preprocessed data (encoding and standardization)
X_tr_prepro = preproc_step.transform(X_train)
X_te_prepro = preproc_step.transform(X_test)
# get the name of the columns after encoding
preproc_cols = X_tr_prepro.columns
# get the name of the columns selected using SelectFromModel
featsel_cols = preproc_cols[featsel_step.get_support()]
# compute the data to be used by the best classifier
X_tr_featsel = X_tr_prepro[featsel_cols]
X_te_featsel = X_te_prepro[featsel_cols]

In [67]:
X_tr_featsel.index.sort_values()

Int64Index([100007, 100009, 100010, 100012, 100022, 100029, 100040, 100041,
            100086, 100130,
            ...
            456094, 456099, 456125, 456129, 456143, 456147, 456165, 456187,
            456191, 456236],
           dtype='int64', name='SK_ID_CURR', length=20000)